In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

import functools

In [2]:
df = pd.read_csv("../data/raw_data.csv")

In [3]:
df_train, df_test = train_test_split(df, test_size = 0.2)

In [4]:
df_train.to_csv("train.csv")
df_test.to_csv("test.csv")

In [5]:
df_train = pd.read_csv("train.csv")

In [6]:
df_train.shape

(20505, 30)

In [7]:
model_file_path = "model.pkl"
params = ["grna_target_sequence", "target_sequence"]

In [8]:
# Drop NA
params.append("cleavage_freq")
for col in params:
    df_train = df_train[df_train[col].notna()]
params.remove("cleavage_freq")

### Get X

In [9]:
# Select
df_X = df_train[params]

# Convert to string
df_X = df_X[params].convert_dtypes()

#Remove dash
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = [
        seq.replace("-", "")
        for seq in df_X[col]
    ]

# Pad
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = df_X[col].str.pad(width = 50, side = "right", fillchar = "X")

# Encode
def encode_nt(nt:str) -> int:
    assert len(nt) == 1
    encoding_dict = {
        'X': 0,
        'A': 0.25,
        'T': 0.5,
        'G': 0.75,
        'C': 1
    }
    return encoding_dict.get(nt.upper())
def encode_seq(seq:str):
    encoding = [
        encode_nt(nt)
        for nt in seq
    ]
    return np.array(encoding)

def encode_col(df, col):
    df[col] = [
        encode_seq(seq)
        for seq in df[col]
    ]
    return df

def encode(df):
    for col in df.select_dtypes(exclude = ["number"]).columns:
        encode_col(df, col)
    return df
df_X = encode(df_X)

# fold
df_X["stacked"] = functools.reduce(lambda x, y: df_X[x].apply(lambda x: x.tolist()) + df_X[y].apply(lambda x: x.tolist()),  df_X.columns)

# Tensorfy
temp = []
for i in df_X["stacked"]:
    temp.append(i)
X = torch.from_numpy(np.array(temp).astype(np.float32))

### Get y

In [10]:
df_y = df_train["cleavage_freq"]
y = torch.Tensor(np.array(df_y).reshape(df_y.shape[0], 1).astype(np.float32))

In [11]:
y

tensor([[7.9289e-21],
        [1.6694e-03],
        [1.4703e-03],
        ...,
        [8.5000e-05],
        [1.6763e-05],
        [2.2296e-03]])

In [12]:
y.dtype

torch.float32

### Train

In [13]:
input_dim = X.shape[1]
X = Variable(X)
y = Variable(y)
model = nn.Linear(input_dim, 1)
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
BATCH_SIZE = 5
EPOCH = 5
torch_dataset = Data.TensorDataset(X, y)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(loader):
        prediction = model(batch_x)
        print(prediction)
        loss = loss_func(prediction, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

tensor([[-0.6929],
        [-0.3087],
        [-0.3570],
        [-0.6310],
        [-0.2901]], grad_fn=<AddmmBackward>)
tensor([[-0.3581],
        [-0.2787],
        [-0.4654],
        [-0.2554],
        [-0.3128]], grad_fn=<AddmmBackward>)
tensor([[-0.4469],
        [-0.5663],
        [-0.4160],
        [-0.2205],
        [-0.2637]], grad_fn=<AddmmBackward>)
tensor([[-0.5334],
        [-0.3369],
        [-0.5690],
        [-0.6038],
        [-0.3800]], grad_fn=<AddmmBackward>)
tensor([[-0.4322],
        [-0.4533],
        [-0.5629],
        [-0.3731],
        [-0.5652]], grad_fn=<AddmmBackward>)
tensor([[-0.7894],
        [-0.3122],
        [-0.4947],
        [-0.4675],
        [-0.3029]], grad_fn=<AddmmBackward>)
tensor([[-0.4202],
        [-0.2595],
        [-0.2457],
        [-0.5518],
        [-0.5475]], grad_fn=<AddmmBackward>)
tensor([[-0.6845],
        [-0.5463],
        [-0.3981],
        [-0.5712],
        [-0.2409]], grad_fn=<AddmmBackward>)
tensor([[-0.3121],
        [-0.3

tensor([[-0.0897],
        [-0.4668],
        [-0.2304],
        [-0.3228],
        [-0.5728]], grad_fn=<AddmmBackward>)
tensor([[-0.5386],
        [-0.4647],
        [-0.4987],
        [-0.2984],
        [-0.2684]], grad_fn=<AddmmBackward>)
tensor([[-0.2035],
        [-0.2228],
        [-0.5642],
        [-0.3380],
        [-0.5175]], grad_fn=<AddmmBackward>)
tensor([[-0.3199],
        [-0.3503],
        [-0.2778],
        [-0.4685],
        [-0.5270]], grad_fn=<AddmmBackward>)
tensor([[-0.5817],
        [-0.1790],
        [-0.6860],
        [-0.2405],
        [-0.4587]], grad_fn=<AddmmBackward>)
tensor([[-0.4907],
        [-0.7119],
        [-0.5291],
        [-0.6501],
        [-0.5642]], grad_fn=<AddmmBackward>)
tensor([[-0.3317],
        [-0.3817],
        [-0.6234],
        [-0.3705],
        [-0.3491]], grad_fn=<AddmmBackward>)
tensor([[-0.4209],
        [-0.2894],
        [-0.6554],
        [-0.5555],
        [-0.4851]], grad_fn=<AddmmBackward>)
tensor([[-0.2811],
        [-0.1

tensor([[-0.2926],
        [-0.5059],
        [-0.2306],
        [-0.5753],
        [-0.2296]], grad_fn=<AddmmBackward>)
tensor([[-0.4887],
        [-0.2033],
        [-0.6258],
        [-0.3079],
        [-0.2525]], grad_fn=<AddmmBackward>)
tensor([[-0.3488],
        [-0.1520],
        [-0.2522],
        [-0.5034],
        [-0.6275]], grad_fn=<AddmmBackward>)
tensor([[-0.2993],
        [-0.5365],
        [-0.6179],
        [-0.6284],
        [-0.4647]], grad_fn=<AddmmBackward>)
tensor([[-0.5675],
        [-0.2749],
        [-0.3070],
        [-0.2347],
        [-0.4779]], grad_fn=<AddmmBackward>)
tensor([[-0.5516],
        [-0.3388],
        [-0.4059],
        [-0.4246],
        [-0.3146]], grad_fn=<AddmmBackward>)
tensor([[-0.3021],
        [-0.4034],
        [-0.4858],
        [-0.5060],
        [-0.3098]], grad_fn=<AddmmBackward>)
tensor([[-0.5961],
        [-0.4745],
        [-0.2323],
        [-0.5200],
        [-0.3604]], grad_fn=<AddmmBackward>)
tensor([[-0.6172],
        [-0.6

tensor([[-0.2450],
        [-0.6012],
        [-0.5225],
        [-0.2663],
        [-0.5963]], grad_fn=<AddmmBackward>)
tensor([[-0.5024],
        [-0.5443],
        [-0.4087],
        [-0.2250],
        [-0.1651]], grad_fn=<AddmmBackward>)
tensor([[-0.5664],
        [-0.3465],
        [-0.1548],
        [-0.3913],
        [-0.1744]], grad_fn=<AddmmBackward>)
tensor([[-0.3872],
        [-0.5847],
        [-0.2618],
        [-0.3936],
        [-0.2750]], grad_fn=<AddmmBackward>)
tensor([[-0.5005],
        [-0.3460],
        [-0.2974],
        [-0.4992],
        [-0.4001]], grad_fn=<AddmmBackward>)
tensor([[-0.5053],
        [-0.4054],
        [-0.2877],
        [-0.1860],
        [-0.1940]], grad_fn=<AddmmBackward>)
tensor([[-0.5582],
        [-0.4403],
        [-0.0823],
        [-0.2896],
        [-0.3372]], grad_fn=<AddmmBackward>)
tensor([[-0.2233],
        [-0.4182],
        [-0.3568],
        [-0.5127],
        [-0.5013]], grad_fn=<AddmmBackward>)
tensor([[-0.5236],
        [-0.5

tensor([[-0.3159],
        [-0.6105],
        [-0.4673],
        [-0.4727],
        [-0.4634]], grad_fn=<AddmmBackward>)
tensor([[-0.2184],
        [-0.4071],
        [-0.3258],
        [-0.1437],
        [-0.3908]], grad_fn=<AddmmBackward>)
tensor([[-0.2744],
        [-0.2461],
        [-0.6663],
        [-0.3479],
        [-0.1991]], grad_fn=<AddmmBackward>)
tensor([[-0.2949],
        [-0.1846],
        [-0.5040],
        [-0.1359],
        [-0.1591]], grad_fn=<AddmmBackward>)
tensor([[-0.1623],
        [-0.3043],
        [-0.2523],
        [-0.6369],
        [-0.2963]], grad_fn=<AddmmBackward>)
tensor([[-0.2956],
        [-0.3079],
        [-0.3748],
        [-0.5549],
        [-0.1664]], grad_fn=<AddmmBackward>)
tensor([[-0.2665],
        [-0.3314],
        [-0.2158],
        [-0.4587],
        [-0.2256]], grad_fn=<AddmmBackward>)
tensor([[-0.3203],
        [-0.3792],
        [-0.4469],
        [-0.1937],
        [-0.5248]], grad_fn=<AddmmBackward>)
tensor([[-0.1281],
        [-0.4

...

tensor([[ 0.0260],
        [-0.1245],
        [-0.1110],
        [ 0.1227],
        [ 0.1965]], grad_fn=<AddmmBackward>)
tensor([[ 0.0540],
        [-0.0686],
        [-0.1031],
        [ 0.1251],
        [ 0.0330]], grad_fn=<AddmmBackward>)
tensor([[ 0.1637],
        [-0.1120],
        [-0.0592],
        [ 0.1049],
        [ 0.1596]], grad_fn=<AddmmBackward>)
tensor([[ 0.1102],
        [ 0.1109],
        [-0.0181],
        [ 0.0802],
        [ 0.0912]], grad_fn=<AddmmBackward>)
tensor([[-0.1315],
        [-0.0228],
        [-0.0963],
        [ 0.0488],
        [-0.1145]], grad_fn=<AddmmBackward>)
tensor([[ 0.1314],
        [-0.1250],
        [ 0.1702],
        [-0.0840],
        [-0.0466]], grad_fn=<AddmmBackward>)
tensor([[ 0.0629],
        [-0.0378],
        [ 0.0153],
        [-0.0856],
        [ 0.0122]], grad_fn=<AddmmBackward>)
tensor([[-0.0748],
        [-0.0101],
        [ 0.1296],
        [ 0.0264],
        [ 0.1834]], grad_fn=<AddmmBackward>)
tensor([[ 0.2381],
        [ 0.0

tensor([[ 0.0406],
        [ 0.1005],
        [-0.0335],
        [-0.0586],
        [-0.0955]], grad_fn=<AddmmBackward>)
tensor([[-0.0823],
        [-0.0695],
        [-0.1795],
        [-0.0389],
        [-0.1211]], grad_fn=<AddmmBackward>)
tensor([[ 0.1565],
        [-0.0306],
        [-0.0665],
        [ 0.0826],
        [-0.0461]], grad_fn=<AddmmBackward>)
tensor([[ 0.0189],
        [ 0.1379],
        [-0.1089],
        [-0.1036],
        [ 0.2382]], grad_fn=<AddmmBackward>)
tensor([[-0.1730],
        [-0.0269],
        [ 0.0807],
        [-0.0988],
        [-0.0576]], grad_fn=<AddmmBackward>)
tensor([[ 0.2004],
        [-0.0507],
        [ 0.0525],
        [-0.1192],
        [ 0.0504]], grad_fn=<AddmmBackward>)
tensor([[-0.1325],
        [-0.0827],
        [ 0.2024],
        [ 0.0999],
        [ 0.0356]], grad_fn=<AddmmBackward>)
tensor([[-0.0215],
        [ 0.1275],
        [ 0.0889],
        [ 0.0113],
        [-0.0914]], grad_fn=<AddmmBackward>)
tensor([[0.1613],
        [0.144

tensor([[ 0.1884],
        [-0.1137],
        [ 0.1137],
        [ 0.1752],
        [ 0.0642]], grad_fn=<AddmmBackward>)
tensor([[ 0.1568],
        [ 0.1966],
        [ 0.1878],
        [-0.0623],
        [ 0.0531]], grad_fn=<AddmmBackward>)
tensor([[ 0.0547],
        [-0.0774],
        [ 0.1059],
        [ 0.0981],
        [-0.1167]], grad_fn=<AddmmBackward>)
tensor([[ 0.1169],
        [-0.1016],
        [-0.0443],
        [ 0.0016],
        [-0.0807]], grad_fn=<AddmmBackward>)
tensor([[ 0.2022],
        [ 0.1376],
        [ 0.1196],
        [ 0.0104],
        [-0.0548]], grad_fn=<AddmmBackward>)
tensor([[-0.1745],
        [-0.1045],
        [-0.1447],
        [-0.0412],
        [ 0.0555]], grad_fn=<AddmmBackward>)
tensor([[ 0.1216],
        [ 0.1647],
        [-0.0211],
        [ 0.1432],
        [ 0.1607]], grad_fn=<AddmmBackward>)
tensor([[-0.0217],
        [-0.1012],
        [ 0.0560],
        [-0.0165],
        [-0.0345]], grad_fn=<AddmmBackward>)
tensor([[-0.0989],
        [-0.0

tensor([[ 0.1746],
        [ 0.1426],
        [-0.1053],
        [ 0.1122],
        [ 0.0037]], grad_fn=<AddmmBackward>)
tensor([[-0.1181],
        [-0.0920],
        [-0.0750],
        [ 0.2591],
        [-0.1109]], grad_fn=<AddmmBackward>)
tensor([[-0.0764],
        [-0.2493],
        [ 0.1239],
        [ 0.0523],
        [ 0.1357]], grad_fn=<AddmmBackward>)
tensor([[ 0.1324],
        [-0.0619],
        [-0.0578],
        [-0.0070],
        [ 0.0932]], grad_fn=<AddmmBackward>)
tensor([[ 0.0773],
        [-0.0785],
        [ 0.1107],
        [ 0.0639],
        [ 0.0408]], grad_fn=<AddmmBackward>)
tensor([[0.0949],
        [0.1535],
        [0.2113],
        [0.0972],
        [0.2214]], grad_fn=<AddmmBackward>)
tensor([[-0.0150],
        [ 0.0947],
        [ 0.0448],
        [-0.0216],
        [ 0.1359]], grad_fn=<AddmmBackward>)
tensor([[-0.1270],
        [ 0.0114],
        [ 0.1454],
        [ 0.1981],
        [ 0.1364]], grad_fn=<AddmmBackward>)
tensor([[-0.0348],
        [-0.0917],

tensor([[ 0.1613],
        [-0.0943],
        [ 0.1526],
        [ 0.1780],
        [ 0.1294]], grad_fn=<AddmmBackward>)
tensor([[0.0169],
        [0.0344],
        [0.1036],
        [0.1608],
        [0.0930]], grad_fn=<AddmmBackward>)
tensor([[ 0.1245],
        [-0.1428],
        [ 0.0995],
        [-0.1085],
        [ 0.0292]], grad_fn=<AddmmBackward>)
tensor([[-0.0824],
        [ 0.1317],
        [-0.2019],
        [-0.0079],
        [ 0.1916]], grad_fn=<AddmmBackward>)
tensor([[ 0.1874],
        [ 0.0320],
        [-0.0861],
        [-0.0391],
        [-0.1663]], grad_fn=<AddmmBackward>)
tensor([[ 0.2137],
        [ 0.1952],
        [-0.0287],
        [-0.0205],
        [ 0.0936]], grad_fn=<AddmmBackward>)
tensor([[0.0689],
        [0.1331],
        [0.3420],
        [0.1172],
        [0.0155]], grad_fn=<AddmmBackward>)
tensor([[-0.1072],
        [-0.0274],
        [ 0.1917],
        [ 0.0919],
        [-0.0275]], grad_fn=<AddmmBackward>)
tensor([[-0.1406],
        [ 0.0866],
    

In [14]:
df_X.columns

Index(['grna_target_sequence', 'target_sequence', 'stacked'], dtype='object')